<a href="https://colab.research.google.com/github/Pooja-0708/GEN-AI/blob/main/GEN_AI(BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMPORT PACKAGES
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [ ]:
# Specify the pre-trained model to use: BERT-base-cased
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(model)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [ ]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I want to go {mask} trip"
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['I', 'want', 'to', 'go', '[MASK]', 'trip']


In [ ]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[ 101,  146, 1328, 1106, 1301,  103, 3868,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3475,  -7.2270,  -7.3749,  ...,  -6.1648,  -5.9542,  -6.3248],
         [ -8.0048,  -8.2208,  -8.0297,  ...,  -6.6677,  -6.5084,  -6.8748],
         [-10.1512,  -9.9800,  -9.9536,  ...,  -6.4909,  -6.4935,  -7.0928],
         ...,
         [ -8.2530,  -7.8892,  -7.6622,  ...,  -6.6485,  -6.8943,  -8.3354],
         [ -7.3362,  -7.1271,  -6.9643,  ...,  -4.8690,  -6.3528,  -6.2122],
         [-12.5144, -12.6587, -12.6163,  ...,  -8.8964, -10.8751, -10.6238]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [ ]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(8, 28996)

In [ ]:
len(tokens)

6

In [ ]:
# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-8.252991  -7.889214  -7.6622005 ... -6.6484823 -6.89427   -8.335351 ]


In [ ]:
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[2.16014229e-09 3.10790638e-09 3.89995058e-09 ... 1.07475895e-08
 8.40555536e-09 1.98935957e-09]


In [ ]:
confidence_scores.sum()

1.0

In [ ]:
# Print the top 5 predicted tokens and their confidence scores
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I want to go a trip 0.35569385
I want to go this trip 0.15046826
I want to go that trip 0.07164778
I want to go the trip 0.05575748
I want to go on trip 0.037876647
